# Set up

In [ ]:
# Import Nessessary Libraries
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)
pd.set_option("display.max_colwidth", None)

# Load data

In [ ]:
# Load Data
df = pd.read_csv("../../data/input/mba.csv")

In [ ]:
# Select needed columns & Rename columns & Re-arrange columns
df = df[['Nhà sản xuất', 'Khách hàng', 'Email khách hàng', 'Ngày', 'Nguồn lưu lượng (Traffic)', 'UTM_source', 'Chi nhánh', 'Loại sản phẩm', 'Tỉnh thành', 'Đơn hàng', 'Sản phẩm', 'Quận huyện vận chuyển', 'Phiên bản', 'T.trạng t.toán', 'T.trạng đ.hàng', 'Phương thức thanh toán', 'Doanh thu', 'Tiền khuyến mãi', 'Doanh thu thuần', 'Tổng hóa đơn', 'Đã thu', 'Số lượng', 'Vận chuyển']]
df.columns = ['manufacturer', 'customer_name', 'email', 'order_date', 'traffic_source', 'utm_source', 'warehouse', 'category_name', 'province', 'order_id', 'product_name', 'district', 'product_type', 'payment_status', 'order_status', 'payment_method', 'amount', 'discount', 'net_amount', 'final_amount', 'received_amount', 'quantity', 'delivery_amount']
df = df[['manufacturer', 'customer_name', 'email', 'order_id', 'order_date', 'product_name', 'product_type', 'category_name', 'quantity', 'amount', 'discount', 'net_amount', 'delivery_amount', 'final_amount', 'received_amount', 'traffic_source', 'utm_source', 'warehouse', 'province', 'district', 'payment_status', 'order_status', 'payment_method']]

# Data Exploration

In [ ]:
# Data Exploration
print('=>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> .info()\n',df.info())
print('=>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> .head()\n',df.head())
print('=>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> .describe()\n',df.describe())
print('=>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> .columns()\n',df.columns)
# Check Duplicated Rows => not found any duplicated rows
print('=>>>>>>>> DF\n',df[df.duplicated()])
# Check Duplicated Order ID => found some duplicated order id
print('=>>>>>>>> Check `order_id`\n', df[df.order_id.duplicated()].head())

In [ ]:
df.traffic_source.unique()

# Pre-processing

## Remove duplicates

In [ ]:
# Pre-processing: Remove duplicated (`order_id`, `product_name`) with amount = 0 (keep all duplicated rows with amount != 0) according to business requirement.
df=df[df.duplicated(subset=['order_id', 'product_name'], keep=False) & (df.amount != 0)]

## Process fields & Cleaning data

## Function

In [ ]:
import unicodedata
def clean_text_field(text, convention=None):
    """
    Clean text field by removing leading/trailing spaces, chuẩn hóa unicode và chuyển đổi theo convention.
    """
    if isinstance(text, str):
        # Chuẩn hóa unicode
        text = unicodedata.normalize('NFC', text)
        if convention == 'title':
            text = text.strip().title()
        elif convention == 'capitalize':
            text = text.strip().capitalize()
        else:
            text = text.strip()
    else:
        text = 'error'  # Xử lý nghiệp vụ sau
    return text


import re
def is_valid_email(email):
    """Kiểm tra email có hợp lệ hay không."""
    if not isinstance(email, str):
        return False
    pattern = r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$'
    return re.match(pattern, email) is not None

def handle_invalid_email(email):
    """Kiểm tra tính hợp lệ của email. Nếu email không hợp lệ, trả về 'error' => Xử lý nghiệp vụ sau."""
    if is_valid_email(email):
        return email
    return 'error'

def mapping_text(text, _mapping_text_dict, remove_special_tail=None, convention=None):
    """ Clean text, replace pattern, mapping value according to business rules. """
    if not isinstance(text, str): # Check if text is not string
        return 'error'
    else:
        if remove_special_tail != None: # Remove special tail if specified
            text = re.sub('--', 'unknown', text) # Replace '--' -> 'unknown' according to business rules.
            pattern=rf'{remove_special_tail}+$'
            text = re.sub(pattern, '', text)
        text = _mapping_text_dict.get(text.lower(), text)
        if convention == 'lower': # Convert to lower case if specified
            return text.lower()
        else:
            return text
    

## Config: Mapping text

In [ ]:
mapping_text_dict = {
    'fb': 'Facebook',
    'facebook': 'Facebook'
}

## Pre-processing each fields

In [ ]:
# Pre-processing: `manufacturer`
df['manufacturer'] = df['manufacturer'].apply(lambda x: clean_text_field(x, convention='title'))

In [ ]:
# Pre-processing: `customer_name`
df['customer_name'] = df['customer_name'].apply(lambda x: clean_text_field(x, convention='title'))

In [ ]:
# Pre-processing: `email`
df['email'] = df['email'].apply(lambda x: clean_text_field(x)) # Clean text field
df['email'] = df['email'].apply(lambda x: handle_invalid_email(x)) # Handle invalid email

In [ ]:
# Pre-processing: `order_date`
df['order_date'] = pd.to_datetime(df['order_date'], format='%d/%m/%Y') # Convert to datetime

In [ ]:
# Pre-processing: `category_name`
df['category_name'] = df['category_name'].apply(lambda x: clean_text_field(x, convention='capitalize')) # Clean text field

In [ ]:
# Pre-processing: `product_name`
df['product_name'] = df['product_name'].apply(lambda x: clean_text_field(x)) # Clean text field
df['product_name'] = df['product_name'].apply(lambda x: mapping_text(x, mapping_text_dict, remove_special_tail='-')) # Mapping giá trị cột 'product_name' theo mapping_text_dict

In [ ]:
# Pre-processing: `product_type`
df['product_type'] = df['product_type'].apply(lambda x: clean_text_field(x)) # Clean text field

In [ ]:
# Pre-processing: `traffic_source`
df['traffic_source'] = df['traffic_source'].apply(lambda x: clean_text_field(x))
# Mapping giá trị cột 'traffic_source' theo mapping_text_dict
df['traffic_source'] = df['traffic_source'].apply(lambda x: mapping_text(x, mapping_text_dict))

In [ ]:
# Pre-processing: `utm_source`
df['utm_source'] = df['utm_source'].apply(lambda x: clean_text_field(x)) # Clean text field
df['utm_source'] = df['utm_source'].apply(lambda x: mapping_text(x, mapping_text_dict, remove_special_tail='-', convention='lower'))

In [ ]:
# Pre-processing: `warehouse`
df['warehouse'] = df['warehouse'].apply(lambda x: clean_text_field(x)) # Clean text field

In [ ]:
# Pre-processing: `province`
df['province'] = df['province'].apply(lambda x: clean_text_field(x)) # Clean text field

In [ ]:
# Pre-processing: `district`
df['district'] = df['district'].apply(lambda x: clean_text_field(x)) # Clean text field

In [ ]:
# Pre-processing: `payment_status`
df['payment_status'] = df['payment_status'].apply(lambda x: clean_text_field(x)) # Clean text field

In [ ]:
# Pre-processing: `order_status`
df['order_status'] = df['order_status'].apply(lambda x: clean_text_field(x)) # Clean text field

In [ ]:
# Pre-processing: `payment_method`
df['payment_method'] = df['payment_method'].apply(lambda x: clean_text_field(x)) # Clean text field

## Convert data source's schema to Dimensional Schema

## Extract data according to Dimensional Schema

In [ ]:
# Dimensional: `customers`
# Identify customer by their email -> Generate `customer_id``
customers_df = df[['email','customer_name']].sort_values(by=['email']).drop_duplicates(subset=(['email'])).reset_index(drop=True)
customers_df['id'] = customers_df.index + 1
# Rename columns
customers_df.columns = ['email', 'customer_name', 'customer_id']
# Re-arrange columns
customers_df = customers_df[['customer_id', 'customer_name', 'email']]

In [ ]:
# Dimensional: `manufacturers`
# Identify manufacturer by their email -> Generate `manufacturer_id``
manufacturers_df = df[['manufacturer']].sort_values(by=['manufacturer']).drop_duplicates().reset_index(drop=True)
manufacturers_df['id'] = manufacturers_df.index + 1
# Rename columns
manufacturers_df.columns = ['manufacturer_name', 'manufacturer_id']
# Re-arrange columns
manufacturers_df = manufacturers_df[['manufacturer_id', 'manufacturer_name']]

In [ ]:
# Dimensional: `categories`
categories_df = df[['category_name']].sort_values(by=['category_name']).drop_duplicates().reset_index(drop=True)
categories_df['id'] = categories_df.index + 1
# Rename columns
categories_df.columns = ['category_name', 'category_id']
# Re-arrange columns
categories_df = categories_df[['category_id', 'category_name']]

In [ ]:
# Dimensional: `products`
products_df = df[['product_name']].sort_values(by=['product_name']).drop_duplicates().reset_index(drop=True)
products_df['id'] = products_df.index + 1
# Rename columns
products_df.columns = ['product_name', 'product_id']
# Re-arrange columns
products_df = products_df[['product_id', 'product_name']]

In [ ]:
# Dimensional: `skus`
skus_df = df[['product_name', 'product_type']].sort_values(by=['product_name', 'product_type']).drop_duplicates().reset_index(drop=True)
skus_df['id'] = skus_df.index + 1

skus_df = skus_df.merge(products_df, on='product_name', how='left') # Map `product_name' to 'product_id'
skus_df['sku_description'] = skus_df['product_name'] + ' | ' + skus_df['product_type']

# Rename columns
skus_df.columns = ['product_name', 'product_type', 'sku_id', 'product_id', 'sku_description']
# Select and Re-arrange columns
skus_df = skus_df[['sku_id', 'sku_description', 'product_id', 'product_name', 'product_type']]

In [ ]:
# Dimensional: `payment_methods`
payment_methods_df = df[['payment_method']].sort_values(by=['payment_method']).drop_duplicates().reset_index(drop=True)
payment_methods_df['id'] = payment_methods_df.index + 1
# Rename columns
payment_methods_df.columns = ['description', 'payment_method']
# Re-arrange columns
payment_methods_df = payment_methods_df[['payment_method', 'description']]

In [ ]:
# Dimensional: `payment_status`
payment_status_df = df[['payment_status']].sort_values(by=['payment_status']).drop_duplicates().reset_index(drop=True)
payment_status_df['id'] = payment_status_df.index + 1
# Rename columns
payment_status_df.columns = ['description', 'payment_status']
# Re-arrange columns
payment_status_df = payment_status_df[['payment_status', 'description']]

In [ ]:
# Dimensional: `order_status`
order_status_df = df[['order_status']].sort_values(by=['order_status']).drop_duplicates().reset_index(drop=True)
order_status_df['id'] = order_status_df.index + 1
# Rename columns
order_status_df.columns = ['description', 'order_status']
# Re-arrange columns
order_status_df = order_status_df[['order_status', 'description']]

In [ ]:
# Dimensional: `warehouses`
warehouses_df = df[['warehouse']].sort_values(by=['warehouse']).drop_duplicates().reset_index(drop=True)
warehouses_df['id'] = warehouses_df.index + 1
# Rename columns
warehouses_df.columns = ['warehouse_name', 'warehouse_id']
# Re-arrange columns
warehouses_df = warehouses_df[['warehouse_id', 'warehouse_name']]

In [ ]:
# Dimensional: `provinces`
provinces_df = df[['province']].sort_values(by=['province']).drop_duplicates().reset_index(drop=True)
provinces_df['id'] = provinces_df.index + 1
# Rename columns
provinces_df.columns = ['province_name', 'province_id']
# Re-arrange columns
provinces_df = provinces_df[['province_id', 'province_name']]

In [ ]:
# Dimensional: `districts`
districts_df = df[['province', 'district']].sort_values(by=['province', 'district']).drop_duplicates().reset_index(drop=True)
districts_df['id'] = districts_df.index + 1
districts_df = districts_df.merge(provinces_df, left_on='province', right_on='province_name', how='left') # Map `province_name' to 'province_id'
# Rename columns
districts_df.columns = ['province', 'district_name', 'district_id', 'province_id', 'province_name']
# Select & Re-arrange columns
districts_df = districts_df[['district_id', 'district_name', 'province_id']]

In [ ]:
# Fact Table: `sale_details_df`
# Map `field_name` to `field_id`

sale_details_df = df

# Map `email` to `customer_id`
sale_details_df = sale_details_df.merge(customers_df[['email', 'customer_id']], on='email', how='left')

# Map (`product_name`,`product_type`) to `sku_id`
sale_details_df = sale_details_df.merge(skus_df[['sku_id', 'product_name', 'product_type']], on=['product_name', 'product_type'], how='left')

# Map `product_name` to `product_id`
sale_details_df = sale_details_df.merge(products_df[['product_id', 'product_name']], on='product_name', how='left')

# Map `category_name` to `category_id`
sale_details_df = sale_details_df.merge(categories_df, on='category_name', how='left')

# Map `manufacturer` to `manufacturer_id`
sale_details_df = sale_details_df.merge(manufacturers_df, left_on='manufacturer', right_on='manufacturer_name', how='left')

# Map `warehouse` to `warehouse_id`
sale_details_df = sale_details_df.merge(warehouses_df, left_on='warehouse', right_on='warehouse_name', how='left')

# Map `payment_method(description)` to `payment_method(id)``
sale_details_df = sale_details_df.merge(payment_methods_df, left_on='payment_method', right_on='description', how='left')

# Map `payment_status(description)` to `payment_status(id)`
sale_details_df = sale_details_df.merge(payment_status_df, left_on='payment_status', right_on='description', how='left')

# Map `order_status(description)` to `order_status(id)`
sale_details_df = sale_details_df.merge(order_status_df, left_on='order_status', right_on='description', how='left')

# Map `province` to `province_id`
sale_details_df = sale_details_df.merge(provinces_df, left_on='province', right_on='province_name', how='left')

# Map `district` to `district_id` (need to map `province_id` first to get `province_id` for mapping `district_id`)
sale_details_df = sale_details_df.merge(districts_df, left_on=['district', 'province_id'], right_on=['district_name', 'province_id'], how='left')

# Generate `id` for `sale_details_df`
sale_details_df['id'] = sale_details_df.index + 1

# Select &  Re-arrange columns
sale_details_df = sale_details_df[['id', 'order_id', 'order_date', 'customer_id', 'sku_id', 'product_id', 'category_id', 'manufacturer_id','warehouse_id', 'payment_method_y', 'payment_status_y','order_status_y', 'district_id', 'province_id', 'quantity', 'amount', 'discount', 'net_amount', 'delivery_amount', 'final_amount', 'received_amount', 'traffic_source', 'utm_source']].reset_index(drop=True)

# Rename columns
sale_details_df.columns = ['id', 'order_id', 'order_date', 'customer_id', 'sku_id', 'product_id', 'category_id', 'manufacturer_id','warehouse_id', 'payment_method', 'payment_status','order_status', 'delivery_district_id', 'delivery_province_id', 'quantity', 'amount', 'discount', 'net_amount', 'delivery_amount', 'final_amount', 'received_amount', 'traffic_source', 'utm_source']

In [ ]:
skus_df.head()

# Connect to DB & Load data

## Connect to DB

In [ ]:
# Import pyodbc and check available drivers
import pyodbc
print(pyodbc.drivers())

# Import DB access information
import sys
sys.path.append('../db_connection/connectors')
from mysql_admin import SERVER, PORT, DATABASE, USERNAME, PASSWORD

In [ ]:
# Connect to DB
conn_str = (
    'DRIVER={MySQL ODBC 9.4 Unicode Driver};'
    f'SERVER={SERVER};'
    f'PORT={PORT};'
    f'DATABASE={DATABASE};'
    f'USER={USERNAME};'
    f'PASSWORD={PASSWORD};'
    'OPTION=3;'
)

conn = pyodbc.connect(conn_str)
print("Connect Successful!")


## Load data

In [ ]:
# Create Table


In [ ]:
# Create Table
import os

# Đọc script tạo bảng từ file SQL
sql_path = os.path.abspath('../../sql/ddl/create_table_replace.sql')
with open(sql_path, 'r', encoding='utf-8') as f:
    create_table_sql = f.read()

# Thực thi script tạo bảng
cursor = conn.cursor()
for statement in create_table_sql.split(';'):
    stmt = statement.strip()
    if stmt:
        try:
            cursor.execute(stmt)
        except Exception as e:
            print(f"Error executing statement: {stmt}\n{e}")
conn.commit()
print("Tables created successfully!")

In [ ]:
# Insert data từ các dataframe vào các bảng
def insert_df_to_db(df, table, conn, if_exists='append'):
    # Sử dụng sqlalchemy để insert nhanh hơn (nếu có)
    try:
        from sqlalchemy import create_engine
        engine_str = f"mysql+pyodbc://{USERNAME}:{PASSWORD}@{SERVER}:{PORT}/{DATABASE}?driver=MySQL+ODBC+9.4+Unicode+Driver"
        engine = create_engine(engine_str)
        df.to_sql(table, con=engine, if_exists=if_exists, index=False)
        print(f"Inserted {len(df)} rows into {table}")
    except ImportError:
        # Nếu không có sqlalchemy, dùng pyodbc thủ công
        cols = ','.join(df.columns)
        placeholders = ','.join(['?'] * len(df.columns))
        for row in df.itertuples(index=False, name=None):
            cursor.execute(f"INSERT INTO {table} ({cols}) VALUES ({placeholders})", row)
        conn.commit()
        print(f"Inserted {len(df)} rows into {table}")

In [ ]:
# Insert lần lượt các bảng dimension và fact
insert_df_to_db(customers_df, 'customers', conn)
insert_df_to_db(manufacturers_df, 'manufacturers', conn)
insert_df_to_db(categories_df, 'categories', conn)
insert_df_to_db(products_df, 'products', conn)
insert_df_to_db(skus_df, 'skus', conn)
insert_df_to_db(payment_methods_df, 'payment_methods', conn)
insert_df_to_db(payment_status_df, 'payment_status', conn)
insert_df_to_db(order_status_df, 'order_status', conn)
insert_df_to_db(warehouses_df, 'warehouses', conn)
insert_df_to_db(provinces_df, 'provinces', conn)
insert_df_to_db(districts_df, 'districts', conn)
insert_df_to_db(sale_details_df, 'sale_details', conn)

# Data Mining

# FP-Growth Algorithm
Cài đặt và chạy thuật toán FP-Growth để phân tích luật kết hợp (association rules) từ dữ liệu giao dịch.

## Giải thích thuật toán FP-Growth và các chỉ số
**FP-Growth** (Frequent Pattern Growth) là một thuật toán khai phá tập mục phổ biến (frequent itemsets) trong dữ liệu giao dịch, thường dùng trong phân tích giỏ hàng (market basket analysis). Thuật toán này giúp tìm ra các nhóm sản phẩm thường được mua cùng nhau mà không cần sinh tất cả các tập con ứng viên như Apriori, do đó nhanh và tiết kiệm bộ nhớ hơn.

### Ý nghĩa các chỉ số:
- **Support (Độ phổ biến):** Tỷ lệ số giao dịch chứa tập mục so với tổng số giao dịch. Support cao nghĩa là tập mục xuất hiện thường xuyên.
- **Confidence (Độ tin cậy):** Xác suất rằng khi khách hàng mua tập mục A thì cũng mua tập mục B. Công thức: confidence(A→B) = support(A∪B) / support(A).
- **Lift (Độ nâng):** Đo lường mức độ liên kết thực sự giữa A và B so với kỳ vọng nếu A và B độc lập. Lift > 1 nghĩa là A và B có mối liên hệ mạnh hơn ngẫu nhiên, lift < 1 là yếu hơn ngẫu nhiên.

### Ứng dụng
- Phát hiện các nhóm sản phẩm thường được mua cùng nhau để đề xuất bán chéo, tối ưu trưng bày sản phẩm, xây dựng chương trình khuyến mãi,...

In [ ]:
# # Cài đặt thư viện mlxtend nếu chưa có
# # !pip install mlxtend

# # Chuẩn bị dữ liệu giao dịch cho FP-Growth
# from mlxtend.frequent_patterns import fpgrowth, association_rules
# from mlxtend.preprocessing import TransactionEncoder

In [ ]:
# # Giả sử mỗi order_id là một giao dịch, lấy danh sách sản phẩm theo order_id
# transactions = df.groupby('order_id')['product_name'].apply(list).tolist()

# # Mã hóa dữ liệu giao dịch
# te = TransactionEncoder()
# te_ary = te.fit(transactions).transform(transactions)
# df_tf = pd.DataFrame(te_ary, columns=te.columns_)

# # Chạy FP-Growth để tìm tập phổ biến
# frequent_itemsets = fpgrowth(df_tf, min_support=0.01, use_colnames=True)
# print('=>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> frequent_itemsets\n',frequent_itemsets.head())

# # Sinh luật kết hợp
# rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)
# print('=>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>> rules\n',rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head())



In [ ]:
# rules[['antecedents', 'consequents', 'antecedent support', 'consequent support', 'support', 'confidence', 'lift']].head()